In [ ]:
!apt update
!apt install -y openjdk-17-jdk-headless
!pip install pathling==7.2.0 deltalake==0.25.4 pandas==2.2.3 polars==1.26.0

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
36 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
# Generate Synthetic Data
!curl -LO https://github.com/synthetichealth/synthea/releases/download/master-branch-latest/synthea-with-dependencies.jar
!java -jar synthea-with-dependencies.jar -s 20250306 -cs 20250306 -r 20250306 -p 50 \
  --exporter.baseDirectory="./synthea/output/bulk" \
  --exporter.fhir.bulk_data="true" \
  --exporter.fhir.export="true" \
  --exporter.hospital.fhir.export="true" \
  --exporter.practitioner.fhir.export="true" \
  --exporter.fhir.included_resources="Practitioner,Organization,Patient,Condition,Observation,Encounter"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  170M  100  170M    0     0  83.2M      0  0:00:02  0:00:02 --:--:--  166M
SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#noProviders for further details.
Scanned 88 modules and 152 submodules.
Loading submodule modules/allergies/allergy_panel.json
Loading submodule modules/allergies/drug_allergy_incidence.json
Loading submodule modules/allergies/environmental_allergy_incidence.json
Loading submodule modules/allergies/food_allergy_incidence.json
Loading submodule modules/allergies/immunotherapy.json
Loading submodule modules/allergies/outgrow_env_allergies.json
Loading submodule modules/allergies/outgrow_food_allergies.json
Loading submodule modules/aller

In [ ]:
from pathling import PathlingContext

pc = PathlingContext.create(
    enable_extensions=True,
    enable_delta=True,
    enable_terminology=False,
    terminology_server_url="http://localhost/not-a-real-server",
)

In [ ]:
from pathling import Expression as exp

data = pc.read.ndjson("./synthea/output/bulk/fhir")

result = data.extract(
    "Patient",
    columns=[
        exp("id", "patient_id"),
        exp("gender", "gender"),
        exp("birthDate", "birth_date"),
        exp("reverseResolve(Condition.subject).code.coding.where(system='http://snomed.info/sct').code", "condition_code"),
        exp("reverseResolve(Condition.subject).code.coding.where(system='http://snomed.info/sct').display", "condition_display"),
    ],
)

result.show(truncate=False)

+------------------------------------+------+----------+--------------+------------------------------------------+
|patient_id                          |gender|birth_date|condition_code|condition_display                         |
+------------------------------------+------+----------+--------------+------------------------------------------+
|a9e56927-089d-387c-3756-ab42ffa637d6|male  |1949-03-06|314529007     |Medication review due (situation)         |
|a9e56927-089d-387c-3756-ab42ffa637d6|male  |1949-03-06|109570002     |Primary dental caries (disorder)          |
|a9e56927-089d-387c-3756-ab42ffa637d6|male  |1949-03-06|73595000      |Stress (finding)                          |
|a9e56927-089d-387c-3756-ab42ffa637d6|male  |1949-03-06|55822004      |Hyperlipidemia (disorder)                 |
|a9e56927-089d-387c-3756-ab42ffa637d6|male  |1949-03-06|266934004     |Transport problem (finding)               |
|a9e56927-089d-387c-3756-ab42ffa637d6|male  |1949-03-06|422650009     |Social is

In [ ]:
data.write.delta("./delta")

In [ ]:
from deltalake import DeltaTable

dt = DeltaTable("./delta/Condition.parquet")
df = dt.to_pandas()
df[["id", "code", "subject"]]

,id,code,subject
0,000b635e-7e55-48c8-0323-8da4e81f7554,"{'id': None, 'coding': [{'id': None, 'system':...","{'id': None, 'reference': 'Patient/26bb7cb9-ba..."
1,0023cff7-186c-1515-5781-c77702b4c419,"{'id': None, 'coding': [{'id': None, 'system':...","{'id': None, 'reference': 'Patient/94642e09-ff..."
2,003badf2-2533-6e45-fca1-0e7bc7c27658,"{'id': None, 'coding': [{'id': None, 'system':...","{'id': None, 'reference': 'Patient/f1dd0ab2-02..."
3,005b7a2a-b54d-ce5c-e09a-26b7ad4fc57a,"{'id': None, 'coding': [{'id': None, 'system':...","{'id': None, 'reference': 'Patient/1561c307-ff..."
4,0079efd1-4b9d-5e7d-90c0-03b559c0b11e,"{'id': None, 'coding': [{'id': None, 'system':...","{'id': None, 'reference': 'Patient/7dc59009-85..."
...,...,...,...
2469,ff8cf01a-3016-14ef-329b-8885cc4a3755,"{'id': None, 'coding': [{'id': None, 'system':...","{'id': None, 'reference': 'Patient/3010759b-b6..."
2470,ffb719db-43a0-94ca-16e2-45598e7ff77d,"{'id': None, 'coding': [{'id': None, 'system':...","{'id': None, 'reference': 'Patient/7af73c09-80..."
2471,ffc4c554-86c8-305e-c050-b72b2c350b18,"{'id': None, 'coding': [{'id': None, 'system':...","{'id': None, 'reference': 'Patient/2ee8a5af-4b..."
2472,ffc535e5-3161-1b6a-0128-a45768c3ff5b,"{'id': None, 'coding': [{'id': None, 'system':...","{'id': None, 'reference': 'Patient/759fcde1-fc..."


In [ ]:
import polars as pl

pl_df = pl.from_dataframe(df)

pl_df.select("code").unnest("code")

_fid,coding,id,text
i64,list[struct[7]],null,str
1073152525,"[{402085576,""73595000"",""Stress (finding)"",null,""http://snomed.info/sct"",null,null}]",null,"""Stress (finding)"""
2127494867,"[{1280739512,""18718003"",""Gingival disease (disorder)"",null,""http://snomed.info/sct"",null,null}]",null,"""Gingival disease (disorder)"""
317601641,"[{771420541,""44054006"",""Diabetes mellitus type 2 (disorder)"",null,""http://snomed.info/sct"",null,null}]",null,"""Diabetes mellitus type 2 (diso…"
665580448,"[{1076429055,""124171000119105"",""Chronic intractable migraine without aura (disorder)"",null,""http://snomed.info/sct"",null,null}]",null,"""Chronic intractable migraine w…"
1562521714,"[{2087917464,""314529007"",""Medication review due (situation)"",null,""http://snomed.info/sct"",null,null}]",null,"""Medication review due (situati…"
…,…,…,…
1086628130,"[{1532145144,""423315002"",""Limited social contact (finding)"",null,""http://snomed.info/sct"",null,null}]",null,"""Limited social contact (findin…"
1667233019,"[{1561605793,""314529007"",""Medication review due (situation)"",null,""http://snomed.info/sct"",null,null}]",null,"""Medication review due (situati…"
1453732321,"[{648092038,""237602007"",""Metabolic syndrome X (disorder)"",null,""http://snomed.info/sct"",null,null}]",null,"""Metabolic syndrome X (disorder…"


In [ ]:
!curl https://install.duckdb.org | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3270  100  3270    0     0   7948      0 --:--:-- --:--:-- --:--:--  7956

*** DuckDB Linux/MacOS installation script, version 1.2.1 ***


         .;odxdl,            
       .xXXXXXXXXKc          
       0XXXXXXXXXXXd  cooo:  
      ,XXXXXXXXXXXXK  OXXXXd 
       0XXXXXXXXXXXo  cooo:  
       .xXXXXXXXXKc          
         .;odxdl,  


############################################################################################# 100.0%

  with a link from                      /root/.duckdb/cli/latest/duckdb

Hint: Append the following line to your shell profile:
export PATH='/root/.duckdb/cli/latest':$PATH


To launch DuckDB now, type
/root/.duckdb/cli/latest/duckdb


In [ ]:
!echo "SELECT * FROM delta_scan('./delta/Observation.parquet');" | /root/.duckdb/cli/latest/duckdb -f /dev/stdin

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬────────────┬──────────────────────┐
│          id          │     id_versioned     │         meta         │ … │      component       │    _fid    │      _extension      │
│       varchar        │       varchar        │ struct(id varchar,…  │   │ struct(id varchar,…  │   int32    │ map(integer, struc…  │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼────────────┼──────────────────────┤
│ 00012b0c-3b8e-03a6…  │ Observation/00012b…  │ {'id': NULL, 'vers…  │ … │ NULL                 │ 1157246233 │ {}                   │
│ 0001545b-bff8-59fa…  │ Observation/000154…  │ NULL                 │ … │ NULL                 │ 1164780179 │ {}                   │
│ 00021d76-c2fe-8fa3…  │ Observation/00021d…  │ {'id': NULL, 'vers…  │ … │ NULL                 │    5789523 │ {}                   │
│ 000224e6-63f5-e625…  │ Observation/000224…  │ {'id': NULL, '

In [ ]:
!echo "SELECT coding.display, COUNT(DISTINCT subject) FROM (SELECT UNNEST(code.coding) AS coding, subject.reference AS subject FROM delta_scan('./delta/Observation.parquet')) GROUP BY coding.display ORDER BY COUNT(DISTINCT subject) DESC;" | /root/.duckdb/cli/latest/duckdb -f /dev/stdin

┌─────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────┐
│                                       display                                       │ count(DISTINCT subject) │
│                                       varchar                                       │          int64          │
├─────────────────────────────────────────────────────────────────────────────────────┼─────────────────────────┤
│ Tobacco smoking status                                                              │                      57 │
│ Respiratory rate                                                                    │                      57 │
│ Leukocytes [#/volume] in Blood by Automated count                                   │                      57 │
│ Erythrocytes [#/volume] in Blood by Automated count                                 │                      57 │
│ Platelets [#/volume] in Blood by Automated count                                    │ 